# SQLite Basics

Date: 2024/04/06, 2024/04/09, 2024/04/21

Salesforce SEの経験あり、ER図やデータモデリングを思い出しながら、SQLite自習。
Salesforceと異なり、生のSQL文法で全て記述していかなければならない。これが今回の自習ポイント。

```
ER diagram

  members                 properties
+-----------+           +---------------+             instruments
| member_id |           | property_id   |           +---------------+
|-----------|    FK     +---------------+           | instrument_id |
| name      | ---------O| member_id     |    FK     +---------------+
| height    |           | instrument_id |O--------- | maker         |
+-----------+           +---------------+           | model         |
                                                    +---------------+
```


In [1]:
import pandas as pd
import hashlib

## SQLite操作練習: テーブル登録

In [2]:
import sqlite3

conn = sqlite3.connect('beatles.db')
conn.execute('PRAGMA foreign_keys=ON')
conn.execute('PRAGMA foreign_key_check')

cur = conn.cursor()

cur.execute('DROP TABLE IF EXISTS properties')
cur.execute('DROP TABLE IF EXISTS members')
cur.execute('DROP TABLE IF EXISTS instruments')

cur.execute('CREATE TABLE members(member_id TEXT PRIMARY KEY, name TEXT, height NUMBER)')
cur.execute('CREATE TABLE instruments(instrument_id TEXT PRIMARY KEY, maker TEXT, model TEXT)')
cur.execute('CREATE TABLE properties(property_id INTEGER PRIMARY KEY AUTOINCREMENT, member_id TEXT,\
instrument_id TEXT, FOREIGN KEY (member_id) REFERENCES members(member_id),\
FOREIGN KEY (instrument_id) REFERENCES instruments(instrument_id),\
UNIQUE(member_id, instrument_id))')

## SQLite操作練習: メンバー登録

In [3]:
members = [['John Lennon', 179], ['Paul McCartney', 180], ['George Harrison', 178], ['Ringo Starr', 170]]
 
MEMBERS = {'name': [row[0] for row in members],
            'height': [row[1] for row in members]}

df_members = pd.DataFrame(MEMBERS)
df_members['member_id'] = df_members['name'].apply(lambda x: hashlib.md5(x.encode('ascii')).hexdigest())

df_members

,name,height,member_id
0,John Lennon,179,fcb9684bfddfe32e9aa62794256fc78d
1,Paul McCartney,180,d8a5441817b1822522ccbd12236dce81
2,George Harrison,178,7811ee1bd89bba41d4cc35a71f8cc65c
3,Ringo Starr,170,602877856e00afc1dcfd1b9fe1701225


In [4]:
def create_members(cursor, name_only=False):
    for idx in range(len(df_members)):
        row = df_members.iloc[idx]
        member_id = row['member_id']
        name = row['name']
        if name_only:
            cursor.execute(f'INSERT INTO members(member_id, name) values("{member_id}", "{name}")')             
        else:
            height = row['height']
            cursor.execute(f'INSERT INTO members(member_id, name, height) values("{member_id}", "{name}", {height})') 

create_members(cur, name_only=True)

In [5]:
def read_members(cursor):
    for record in cur.execute('SELECT * FROM members'):
        print(record)

read_members(cur)

('fcb9684bfddfe32e9aa62794256fc78d', 'John Lennon', None)
('d8a5441817b1822522ccbd12236dce81', 'Paul McCartney', None)
('7811ee1bd89bba41d4cc35a71f8cc65c', 'George Harrison', None)
('602877856e00afc1dcfd1b9fe1701225', 'Ringo Starr', None)


In [6]:
def update_members(cursor):
    for idx in range(len(df_members)):
        row = df_members.iloc[idx]
        member_id = row['member_id']
        height = row['height']
        cursor.execute(f'UPDATE members SET height={height} WHERE member_id="{member_id}"')             

update_members(cur)

In [7]:
read_members(cur)

('fcb9684bfddfe32e9aa62794256fc78d', 'John Lennon', 179)
('d8a5441817b1822522ccbd12236dce81', 'Paul McCartney', 180)
('7811ee1bd89bba41d4cc35a71f8cc65c', 'George Harrison', 178)
('602877856e00afc1dcfd1b9fe1701225', 'Ringo Starr', 170)


In [8]:
conn.commit()

In [9]:
def delete_members(cursor):
    for idx in range(len(df_members)):
        row = df_members.iloc[idx]
        name = row['name']
        cursor.execute(f'DELETE from members WHERE name="{name}"')
        read_members(cur)
        print('---')

delete_members(cur)

('d8a5441817b1822522ccbd12236dce81', 'Paul McCartney', 180)
('7811ee1bd89bba41d4cc35a71f8cc65c', 'George Harrison', 178)
('602877856e00afc1dcfd1b9fe1701225', 'Ringo Starr', 170)
---
('7811ee1bd89bba41d4cc35a71f8cc65c', 'George Harrison', 178)
('602877856e00afc1dcfd1b9fe1701225', 'Ringo Starr', 170)
---
('602877856e00afc1dcfd1b9fe1701225', 'Ringo Starr', 170)
---
---


In [10]:
conn.commit()

In [11]:
read_members(cur)

## SQLite操作練習: 楽器登録

全部登録するときりがないので適当に。

In [12]:
instruments = [['Gibson', 'J-160E'], ['Martin', 'D-28'], ['Gibson', 'J-200'], ['Rickenbacker', '325'], ['Höfner', 'Club 40'],
              ['Fender', 'Stratocaster'], ['Höfner', 'model 500/1'], ['Epiphone' ,'Casino'], ['Ludwig', '']]

INSTRUMENTS = {'maker': [row[0] for row in instruments], 
            'model': [row[1] for row in instruments]}
df_instruments = pd.DataFrame(INSTRUMENTS)
df_instruments['instrument_id'] = (df_instruments['maker'] + df_instruments['model']).apply(lambda x: hashlib.md5(x.encode('utf-8')).hexdigest())

df_instruments

,maker,model,instrument_id
0,Gibson,J-160E,f327bb219e7a9863ff0be90fb29a7891
1,Martin,D-28,f8ccb4ce4605cec8cc383d286bd8b7ef
2,Gibson,J-200,a351dfcc8c58c660d0652cd5ca81df8d
3,Rickenbacker,325,9dfe456824511fec363696e174ce42be
4,Höfner,Club 40,485d3791969e4d2fe5dc7bcd70287a9a
5,Fender,Stratocaster,9e56e6a849989f39562ce718a4472bb8
6,Höfner,model 500/1,63e6ec26f4811e5f8fc25bffb238ad03
7,Epiphone,Casino,8567844e7c65db4f3bf83063f8fb473a
8,Ludwig,,d3b8642bd00168efb8526ebcb130f00b


In [13]:
def create_instruments(cursor):
    for idx in range(len(df_instruments)):
        row = df_instruments.iloc[idx]
        instrument_id = row['instrument_id']
        maker = row['maker']
        model = row['model']
        cursor.execute(f'INSERT INTO instruments(instrument_id, maker, model) values("{instrument_id}", "{maker}", "{model}")') 

create_instruments(cur)

In [14]:
conn.commit()

In [15]:
def read_instruments(cursor):
    for record in cur.execute('SELECT * FROM instruments'):
        print(record)

read_instruments(cur)

('f327bb219e7a9863ff0be90fb29a7891', 'Gibson', 'J-160E')
('f8ccb4ce4605cec8cc383d286bd8b7ef', 'Martin', 'D-28')
('a351dfcc8c58c660d0652cd5ca81df8d', 'Gibson', 'J-200')
('9dfe456824511fec363696e174ce42be', 'Rickenbacker', '325')
('485d3791969e4d2fe5dc7bcd70287a9a', 'Höfner', 'Club 40')
('9e56e6a849989f39562ce718a4472bb8', 'Fender', 'Stratocaster')
('63e6ec26f4811e5f8fc25bffb238ad03', 'Höfner', 'model 500/1')
('8567844e7c65db4f3bf83063f8fb473a', 'Epiphone', 'Casino')
('d3b8642bd00168efb8526ebcb130f00b', 'Ludwig', '')


## SQLite操作練習: メンバーと楽器の間のリレーションを生成

In [16]:
read_members(cur)

In [17]:
create_members(cur)

In [18]:
read_members(cur)

('fcb9684bfddfe32e9aa62794256fc78d', 'John Lennon', 179)
('d8a5441817b1822522ccbd12236dce81', 'Paul McCartney', 180)
('7811ee1bd89bba41d4cc35a71f8cc65c', 'George Harrison', 178)
('602877856e00afc1dcfd1b9fe1701225', 'Ringo Starr', 170)


In [19]:
read_instruments(cur)

('f327bb219e7a9863ff0be90fb29a7891', 'Gibson', 'J-160E')
('f8ccb4ce4605cec8cc383d286bd8b7ef', 'Martin', 'D-28')
('a351dfcc8c58c660d0652cd5ca81df8d', 'Gibson', 'J-200')
('9dfe456824511fec363696e174ce42be', 'Rickenbacker', '325')
('485d3791969e4d2fe5dc7bcd70287a9a', 'Höfner', 'Club 40')
('9e56e6a849989f39562ce718a4472bb8', 'Fender', 'Stratocaster')
('63e6ec26f4811e5f8fc25bffb238ad03', 'Höfner', 'model 500/1')
('8567844e7c65db4f3bf83063f8fb473a', 'Epiphone', 'Casino')
('d3b8642bd00168efb8526ebcb130f00b', 'Ludwig', '')


In [20]:
# Reference: https://en.wikipedia.org/wiki/List_of_the_Beatles%27_instruments
instruments = {'John Lennon': [['Gibson', 'J-160E'], ['Martin', 'D-28'], ['Rickenbacker', '325'], ['Höfner', 'Club 40'], ['Fender', 'Stratocaster']],
               'George Harrison': [['Gibson', 'J-160E'], ['Gibson', 'J-200'], ['Fender', 'Stratocaster']],
               'Paul McCartney': [['Epiphone' ,'Casino'], ['Höfner', 'model 500/1']],
               'Ringo Starr': [['Ludwig', '']]}

def create_properties(cursor):
    for name, items in instruments.items():
        print(f'--- {name} ---')
        member_id = cursor.execute(f'SELECT member_id FROM members WHERE name="{name}"').fetchone()[0]
        for item in items:
            maker = item[0]
            model = item[1]
            instrument_id = cursor.execute(f'SELECT instrument_id FROM instruments WHERE maker="{maker}" AND model="{model}"').fetchone()[0]
            print(f'{name} {maker} {model} {member_id} {instrument_id}')
            cursor.execute(f'INSERT OR IGNORE INTO properties(member_id, instrument_id) values("{member_id}", "{instrument_id}")')

create_properties(cur)

--- John Lennon ---
John Lennon Gibson J-160E fcb9684bfddfe32e9aa62794256fc78d f327bb219e7a9863ff0be90fb29a7891
John Lennon Martin D-28 fcb9684bfddfe32e9aa62794256fc78d f8ccb4ce4605cec8cc383d286bd8b7ef
John Lennon Rickenbacker 325 fcb9684bfddfe32e9aa62794256fc78d 9dfe456824511fec363696e174ce42be
John Lennon Höfner Club 40 fcb9684bfddfe32e9aa62794256fc78d 485d3791969e4d2fe5dc7bcd70287a9a
John Lennon Fender Stratocaster fcb9684bfddfe32e9aa62794256fc78d 9e56e6a849989f39562ce718a4472bb8
--- George Harrison ---
George Harrison Gibson J-160E 7811ee1bd89bba41d4cc35a71f8cc65c f327bb219e7a9863ff0be90fb29a7891
George Harrison Gibson J-200 7811ee1bd89bba41d4cc35a71f8cc65c a351dfcc8c58c660d0652cd5ca81df8d
George Harrison Fender Stratocaster 7811ee1bd89bba41d4cc35a71f8cc65c 9e56e6a849989f39562ce718a4472bb8
--- Paul McCartney ---
Paul McCartney Epiphone Casino d8a5441817b1822522ccbd12236dce81 8567844e7c65db4f3bf83063f8fb473a
Paul McCartney Höfner model 500/1 d8a5441817b1822522ccbd12236dce81 63e6ec26

In [21]:
def test_fk_constraints(cursor):
    member_id = "000011112222"
    instrument_id = "333344445555"
    try:
        cursor.execute(f'INSERT INTO properties(member_id, instrument_id) values("{member_id}", "{instrument_id}")')
    except Exception as err:
        print(err)

test_fk_constraints(cur)

FOREIGN KEY constraint failed


In [22]:
conn.commit()
conn.close()

## SQLite操作練習: リレーション確認

In [23]:
conn = sqlite3.connect('beatles.db')
cur = conn.cursor()

In [24]:
def dump_properties(member_name):
    member_id = cur.execute(f'SELECT member_id FROM members WHERE name="{member_name}"').fetchone()[0]
    properties = cur.execute(f'SELECT instrument_id FROM properties WHERE member_id="{member_id}"').fetchall()
    
    for record in properties:
        instrument_id = record[0]
        instrument = cur.execute(f'SELECT maker, model from instruments where instrument_id="{instrument_id}"')
        for field in instrument:
            print(field)

In [25]:
dump_properties('John Lennon')

('Höfner', 'Club 40')
('Rickenbacker', '325')
('Fender', 'Stratocaster')
('Gibson', 'J-160E')
('Martin', 'D-28')


In [26]:
dump_properties('Paul McCartney')

('Höfner', 'model 500/1')
('Epiphone', 'Casino')


In [27]:
dump_properties('George Harrison')

('Fender', 'Stratocaster')
('Gibson', 'J-200')
('Gibson', 'J-160E')


In [28]:
dump_properties('Ringo Starr')

('Ludwig', '')


In [29]:
conn.close()